In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../")

import pandas as pd
import time
from azureml.core import Workspace
ws = Workspace.from_config()

kv=ws.get_default_keyvault()


C:\Users\nserafino\Anaconda3\envs\mlopsenv\lib\site-packages\cryptography\hazmat\_der.py:9: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes, int_to_bytes
C:\Users\nserafino\Anaconda3\envs\mlopsenv\lib\site-packages\pkg_resources\__init__.py:122: PkgResourcesDeprecationWarning: win32 is an invalid version and will not be supported in a future release
  warnings.warn(
Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception The 'applicationinsights' distribution was not found and is required by azureml-telemetry.
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.scriptrun = azureml.core.script_run:ScriptRun._from_run_dto with exception (urllib3 1.26.8 (c:\users\nserafino\anaconda3\envs\mlopsenv\lib\site-packages), Requirement.parse('urllib3<=1.26.7,>=1.23')).
Failure while loa

#### Prerequisite
If you use the provision feature, then just need a workspace ws object
If you need to create the cluster manually
    1. Create a service principal and secret (SP)
    2. Provision an ADX cluster and create a DB
    3. Add the SP to be contributor of the cluster
    4. pip install following packages: pip install --upgrade azure-mgmt-kusto azure-kusto-ingest azure-kusto-data azure-identity 



#### Provisioning resource


The following code provision ADX cluster and register neccessary information at the workspace's keyvault

In [ ]:
# #Option to set parameters to a custom ADX cluster
# from monitoring import KV_SP_ID, KV_SP_KEY, KV_ADX_DB, KV_ADX_URI, KV_TENANT_ID
# tenant_id = "72f988bf-86f1-41af-91ab-2d7cd011db47"
# #Application ID
# client_id = "af883abf-89dd-4889-bdb3-1ee84f68465e"
# #Client Secret, set it at your WS' keyvault with key name same as your client_id
# client_secret = kv.get_secret(client_id)
# subscription_id = "0e9bace8-7a81-4922-83b5-d995ff706507"

# cluster_uri = "https://adx02.westus2.kusto.windows.net" #URL of the ADX Cluster
# kv.set_secret(KV_SP_ID,client_id)
# kv.set_secret(KV_SP_KEY,client_secret)
# kv.set_secret(KV_ADX_DB,"db01")
# kv.set_secret(KV_ADX_URI,cluster_uri)
# kv.set_secret(KV_TENANT_ID,tenant_id)


In [ ]:
from monitoring.management import provision
provision(ws)

#### Test Ingestion


In [3]:
from monitoring.data_collector import Online_Collector
table_name = "isd_weather_test4" #new dataset

sample_pd_data = pd.read_parquet("data/test_data.parquet").head(10)
sample_pd_data['timestamp'] = sample_pd_data['datetime']
sample_pd_data.drop(['datetime'], inplace=True, axis=1)

online_collector = Online_Collector(table_name,ws=ws)


In [4]:
online_collector.cluster_uri

'https://adx02.westus2.kusto.windows.net'

In [302]:
online_collector.stream_collect_df(sample_pd_data)

AttributeError: 'Online_Collector' object has no attribute 'stream_collect_df'

In [303]:
online_collector.batch_collect(sample_pd_data)

### Streaming Ingestion and Real Time Visualization

In [282]:

# Ingest streaming data  asynchronously with internal buffering mechanism to lower impact to main scoring thread
streaming_table_name="streaming_test"
streaming_collector = Online_Collector(streaming_table_name,ws=ws)

import random
streaming_collector.start_logging_df(buffer_time=2, batch_size=10)

for run_id in ["r000001", "r000002", "r000003", "r000004", "r000005"]:
    for i in range(1000):
        for lr in ["0.001", "0.002"]:
            df = pd.DataFrame({ "timestamp":pd.to_datetime('today'), "lr":[lr],"metric1":[random.uniform(3,50)] })
            streaming_collector.stream_collect_df_queue(df)
# streaming_collector.stop_logging()



TypeError: __init__() got an unexpected keyword argument 'ws'

In [335]:
dftest = df.append(pd.Series(data=["10 days", [1,2,3]], index=["day", "avg_temperature"]), ignore_index=True)
dftest

C:\Users\nserafino\AppData\Local\Temp\ipykernel_115072\1097353187.py:1: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



,day,avg_temperature
0,2 days 00:00:00,0.1
1,0 days 00:00:00,0.5
2,1 days 00:00:00,-2.25
3,5 days 00:00:00,1.3
4,6 days 00:00:00,1.0
5,4 days 00:00:00,-9.5
6,10 days,"[1, 2, 3]"


In [334]:
dftest.avg_temperature.notnull().sum()

6

In [337]:
dfread = pd.read_csv("C:/Users/nserafino/Downloads/export - 2022-05-05T140742.247.csv")
dfread

,categorical_feature,base_categorical_feature_value,base_dcount,period2,categorical_feature1,target_categorical_feature_value,target_dcount,basearraylength,targetarraylength
0,usaf,"[711740,122260,749484,724673,722140,603900,584...",15,2013-04-13T00:00:00Z,usaf,"[711740,722140,711740,225500,584570,747946,724...",15,1000,381
1,usaf,"[711740,122260,749484,724673,722140,603900,584...",15,2013-05-13T00:00:00Z,usaf,"[122260,749484,724673,603900,584570,722010,749...",15,1000,619
2,wban,"[99999,99999,395,93009,93805,99999,99999,99999...",8,2013-04-13T00:00:00Z,wban,"[99999,93805,99999,99999,99999,12886,13733,395...",8,1000,381
3,wban,"[99999,99999,395,93009,93805,99999,99999,99999...",8,2013-05-13T00:00:00Z,wban,"[99999,395,93009,99999,99999,12836,395,99999,9...",8,1000,619


In [359]:
from scipy.spatial import distance
import random

In [362]:
n = dfread.shape[0]
distance1 = []
for i in range(n):
    base_features = [int(x.strip("[").strip("]")) for x in dfread["base_categorical_feature_value"][i].split(",")]
    target_features = [int(x.strip("[").strip("]")) for x in dfread["target_categorical_feature_value"][i].split(",")]
    if len(target_features) > len(base_features):
        target_features = random.sample(target_features, len(base_features))
    elif len(target_features) < len(base_features):
        base_features = random.sample(base_features, len(target_features))

    distance1.append(distance.euclidean(target_features, base_features))
    #     print(type(int(j.strip("[").strip("]"))))



In [363]:
distance1

[6295999.0974449795, 8439608.529132202, 1165274.6175211233, 1452404.0810938256]

In [307]:
from monitoring.query import RT_Visualization
rt_viz =RT_Visualization(streaming_table_name,ws)
rt_viz.scatter(max_records=200, ago='12h',groupby='lr', y_metric='metric1',x_metric='timestamp')

### Anomaly Detection

In [ ]:
from monitoring.query import KustoQuery
query = KustoQuery(streaming_table_name, ws)

query.anomaly_detection(min_t="4/26/2022 10:08:45", max_t="4/23/2022 10:10:00", step="0.5s", metric="metric1", agg="avg", ts_col="timestamp", groupby="lr", sensitivity=0.1, filter="0.001")


### Drift Query

In [5]:
from monitoring.drift_analysis import Drift_Analysis
drift_analysis =Drift_Analysis(ws)


In [6]:
drift_analysis.sample_data_points("StormEvents", col_name="EventId", start_date = "01/01/2007", horizon=30 )

,EventId
0,13208
1,23358
2,23357
3,9494
4,9488
...,...
3261,9963
3262,12466
3263,9969
3264,11485


In [390]:
df_output = drift_analysis.analyze_drift(limit=30000000,base_table_name = 'ISDWeather',tgt_table_name='ISDWeather', base_dt_from='2013-04-13', base_dt_to='2014-05-13', tgt_dt_from='2013-04-13', tgt_dt_to='2014-05-13', bin='30d')


In [ ]:
df_output

In [13]:
from jupyter_dash import JupyterDash
import dash
from datetime import date
from dateutil import parser
import plotly.express as px
from dash import Dash, dcc, html, Input, Output, State
import plotly.graph_objects as go
import json
from monitoring.drift_analysis import Drift_Analysis
drift_analysis =Drift_Analysis(ws)

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

tables_list = drift_analysis.list_tables()


app.layout = html.Div([
    dcc.Tabs([
        dcc.Tab(label='Dataset Drift', children=[
               html.Div([

        html.Div([
            dcc.Dropdown(
                tables_list,
                tables_list[0],
                id='table_option',
            )]),
            
            html.Div([ dcc.Dropdown(
                id='columns_option',
                multi=True
            )],),

        ]),
        dcc.Graph(id='graph_overview'),]),
        dcc.Tab(label='Feature Analysis', children=[
    html.Div([

        html.Div([
            dcc.Dropdown(
                tables_list,
                tables_list[0],
                id='tables',
            ),
            dcc.DatePickerRange(
            id='base_timeline',start_date_placeholder_text ="base_start_date",end_date_placeholder_text ="base_end_date"
            

            ),
            dcc.DatePickerRange(
            id='target_timeline',start_date_placeholder_text ="target_start_date",end_date_placeholder_text ="target_end_date"

            ),
            html.Button('Prepare data', id='prepare_data', n_clicks=0),
            dcc.Store(id='intermediate-value')
        ], style={'width': '20%', 'display': 'inline-block'}),

        html.Div(

            [ dcc.Dropdown(
                id='columns'
            ),
            dcc.Dropdown(id='metrics'),
            dcc.Store(id='column-name-type')




        ], style={'width': '20%', 'float': 'right', 'display': 'inline-block'}),
        dcc.Graph(id='graph')

    ]),
    html.Div([

    dcc.Graph(id='detail_graph')
    ])

])
    ])])

# overview tab
@app.callback(
    Output('columns_option', 'options'),
    Input('table_option', 'value'))
def set_columns_options(table_name):
    return drift_analysis.list_table_columns(table_name)['AttributeName'].values

@app.callback(
    Output('columns_option', 'value'),
    Input('columns_option', 'options'))
def set_columns_value(available_options):
    return available_options

# feature analysis tab
@app.callback(
    Output('columns', 'options'),
    Input('tables', 'value'))
def set_columns_options(table_name):
    return drift_analysis.list_table_columns(table_name)['AttributeName'].values
@app.callback(
    Output('column-name-type', 'data'),
    Input('tables', 'value'))
def set_columns_options(table_name):
    return drift_analysis.list_table_columns(table_name).to_json(date_format='iso', orient='split')

@app.callback(
    Output('columns', 'value'),
    Input('columns', 'options'))
def set_columns_value(available_options):
    return available_options[0]

@app.callback(
    Output('metrics', 'options'),
    Input('columns', 'value'),
    Input('column-name-type', 'data'),
)
def set_metric_options(column, column_dict):
    column_df = pd.read_json(column_dict, orient='split')
    column_type = str(column_df[column_df['AttributeName']==column]['AttributeType'].values[0])
    if column_type=='StringBuffer':
        return ['dcount', 'euclidean']
    elif column_type=='DateTime':
        return ['NA']
    else:
        return ['max', 'min', "mean", "wasserstein"]
@app.callback(
    Output('metrics', 'value'),
    Input('metrics', 'options'))
def set_columns_value(available_options):
    return available_options[0]


@app.callback(
    [Output('base_timeline', 'min_date_allowed'),Output('base_timeline', 'max_date_allowed'),Output('base_timeline', 'initial_visible_month'),Output('base_timeline', 'start_date'), Output('base_timeline', 'end_date'),
    Output('target_timeline', 'min_date_allowed'),Output('target_timeline', 'max_date_allowed'),Output('target_timeline', 'initial_visible_month'),Output('target_timeline', 'start_date'), Output('target_timeline', 'end_date')],
    Input('tables', 'value'))
def set_columns_options(table_name):
    time_range =drift_analysis.get_time_range(table_name)

    start_date = parser.parse(time_range[0]).replace(microsecond=0, second=0, minute=0)
    end_date = parser.parse(time_range[1]).replace(microsecond=0, second=0, minute=0)
    
    return [start_date, end_date,end_date, start_date, end_date,start_date, end_date,end_date, start_date, end_date]
@app.callback(
    Output('intermediate-value', 'data'),
    State('tables', 'value'),
    State('base_timeline', 'start_date'),
    State('base_timeline', 'end_date'), 
    State('target_timeline', 'start_date'),
    State('target_timeline', 'end_date'), 
    Input('prepare_data', 'n_clicks'),
        prevent_initial_call=True

    
    )
def calculate(table_name, base_start_date, base_end_date,target_start_date, target_end_date,n_clicks):
            # self,base_table_name,target_table_name, base_dt_from, base_dt_to, target_dt_from, target_dt_to, bin, limit=10000000, concurrent_run=True)
    query_df = drift_analysis.analyze_drift(limit=100000,base_table_name = f'{table_name}', target_table_name=f'{table_name}', base_dt_from=f'{base_start_date}', base_dt_to=f'{base_end_date}', target_dt_from=f'{target_start_date}', target_dt_to=f'{target_end_date}', bin='30d')
    return query_df.to_json(date_format='iso', orient='split')


@app.callback(
    Output('graph', 'figure'),
    Input('columns', 'value'),
    Input('metrics', 'value'),
    Input('intermediate-value', 'data'),
    prevent_initial_call=True


    )
def update_figure(column,metric, jsonified_cleaned_data):
    x_values =[0]
    y_values =[0]
    if jsonified_cleaned_data is not None:
        dff = pd.read_json(jsonified_cleaned_data, orient='split')
        dff.sort_values("target_start_date", inplace=True)
        if metric != "wasserstein" and metric != "euclidean" and metric !='NA':
            metric = "target_"+metric
        if dff[dff['numeric_feature']==column].shape[0]>0:
            y_values = dff[dff['numeric_feature']==column][metric]
            x_values= dff[dff['numeric_feature']==column]["target_start_date"]
        elif dff[dff['categorical_feature']==column].shape[0]>0:
            y_values = dff[dff['categorical_feature']==column][metric]

            x_values = dff[dff['categorical_feature']==column]["target_start_date"]


    fig = go.Figure([go.Scatter(x=x_values, y=y_values, name=f"{column}")])


    return fig
@app.callback(
    Output('detail_graph', 'figure'),
    Input('graph', 'clickData'),   
    Input('tables', 'value'),
    Input('columns', 'value'),


)
def drilldown(click_data,table, column):

    # using callback context to check which input was fired
    ctx = dash.callback_context
    trigger_id = ctx.triggered[0]["prop_id"].split(".")[0]

    if trigger_id == 'graph':

        if click_data is not None:
            start_date = click_data['points'][0]['x']
            output = drift_analysis.sample_data_points(table,column,start_date,horizon=30)
            output.dropna(inplace=True)
            fig = px.histogram(output,x=column )
            return fig

    else:
        fig = go.Figure(data=[go.Histogram(x=[0] )])
        return fig

app.run_server()



Dash app running on http://127.0.0.1:8050/
